In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import folium

warnings.filterwarnings(action='ignore')
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.options.display.float_format="{:.5f}".format
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
data = pd.read_csv('data/data.csv')
#submission = pd.read_csv('data/submission.csv')
target = pd.read_csv('data/target.csv')
#student = pd.read_csv('data/인천_학령인구_추계.csv', encoding = 'cp949')

In [3]:
data.head()

,수요지 id,X,Y,초등학령인구
0,C1,126.37150,37.46378,1
1,C2,126.47950,37.49578,46
2,C3,126.48150,37.49978,36
3,C4,126.48250,37.49578,81
4,C5,126.48350,37.49778,25


In [4]:
data.columns = ['수요지 id', '수요_X', '수요_Y', '초등학령인구']

In [5]:
data.head()

,수요지 id,수요_X,수요_Y,초등학령인구
0,C1,126.37150,37.46378,1
1,C2,126.47950,37.49578,46
2,C3,126.48150,37.49978,36
3,C4,126.48250,37.49578,81
4,C5,126.48350,37.49778,25


In [6]:
target.head()

,학교 id,X,Y
0,F1,126.54392,37.46729
1,F2,126.56873,37.49630
2,F3,126.49277,37.48443
3,F4,126.62968,37.49578
4,F5,126.52829,37.45028


In [7]:
target.columns = ['학교 id', '공급_X', '공급_Y']

In [8]:
target.head()

,학교 id,공급_X,공급_Y
0,F1,126.54392,37.46729
1,F2,126.56873,37.49630
2,F3,126.49277,37.48443
3,F4,126.62968,37.49578
4,F5,126.52829,37.45028


In [9]:
sum_df = pd.DataFrame()

# 반복문을 통해 df1 각 행에 df2를 붙이기
for _, row in target.iterrows():
    temp_df = pd.concat([pd.DataFrame(row).T] * len(data), ignore_index=True)
    temp_df = temp_df.join(data)
    sum_df = pd.concat([sum_df, temp_df])

sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,초등학령인구
0,F1,126.54392,37.46729,C1,126.37150,37.46378,1
1,F1,126.54392,37.46729,C2,126.47950,37.49578,46
2,F1,126.54392,37.46729,C3,126.48150,37.49978,36
3,F1,126.54392,37.46729,C4,126.48250,37.49578,81
4,F1,126.54392,37.46729,C5,126.48350,37.49778,25
...,...,...,...,...,...,...,...
306,F50,126.50114,37.51461,C307,126.65150,37.48278,2
307,F50,126.50114,37.51461,C308,126.65250,37.47278,12
308,F50,126.50114,37.51461,C309,126.65250,37.47978,1
309,F50,126.50114,37.51461,C310,126.65250,37.48078,1


In [10]:
sum_df = sum_df.drop('초등학령인구', axis = 1)

In [11]:
sum_df['위도_범위'] = sum_df.apply(lambda row: sorted([row['수요_Y'], row['공급_Y']]), axis=1)

In [12]:
sum_df['경도_범위'] = sum_df.apply(lambda row: sorted([row['수요_X'], row['공급_X']]), axis=1)

In [13]:
sum_df.head()

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]"
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]"
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]"
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]"
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]"


In [14]:
sum_df['공장_개수'] = 0

In [15]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,공장_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",0
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",0
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",0
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",0
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",0
...,...,...,...,...,...,...,...,...,...
306,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",0
307,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
308,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
309,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0


In [16]:
sum_df.reset_index(drop=True, inplace=True)

In [17]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,공장_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",0
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",0
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",0
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",0
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",0
...,...,...,...,...,...,...,...,...,...
15545,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",0
15546,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
15547,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
15548,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0


In [18]:
warning = pd.read_csv('data/인천광역시_공장등록 현황_중구,동구.csv', encoding = 'cp949')

In [19]:
warning.head()

,순번,단지명,회사명,시군구,공장대표주소(도로명),Latitude,Longitude,전화번호,종업원수,대표업종번호,업종번호,업종명,생산품
0,272,NaN,(주)대서양푸드,중구,"인천광역시 중구 연안부두로75번길 4, 2층1호 (항동7가)",37.45266,126.59869,NaN,0,10219,10219,기타 수산동물 가공 및 저장 처리업,수산물
1,273,NaN,(주)에스와이컴퍼니,중구,"인천광역시 중구 연안부두로75번길 4, 2층2호 (항동7가)",37.45266,126.59869,NaN,1,10219,10219,기타 수산동물 가공 및 저장 처리업,"냉동삼치, 냉동임연수"
2,274,NaN,(주)에스지개발,중구,인천광역시 중구 축항대로290번길 41 (신흥동3가),37.44443,126.62566,NaN,5,17902,17902,위생용 종이제품 제조업,N커버(일회용좌변기시트커버)
3,275,NaN,동아테크솔루션,중구,인천광역시 중구 제물량로 37 (신흥동3가),37.46368,126.63613,NaN,4,26529,"26421, 26529",기타 음향기기 제조업 외 1 종,"스피커, 돔카메라"
4,276,NaN,선두수산,중구,인천광역시 중구 서해대로180번길 49 (신흥동3가),37.44263,126.62701,NaN,13,10213,10213,수산동물 냉동품 제조업,냉동수산물


In [20]:
from tqdm import tqdm
import time

In [21]:
for i in tqdm(range(len(sum_df))):
    time.sleep(0.01)
    for j in range(len(warning)):
        if (sum_df.iloc[i, 6][0] <= warning.iloc[j,5] <= sum_df.iloc[i, 6][1]) and (sum_df.iloc[i, 7][0] <= warning.iloc[j,6] <= sum_df.iloc[i, 7][1]):
            sum_df.at[i, '공장_개수'] += 1

100%|████████████████████████████████████████████████████████████████████████████| 15550/15550 [08:06<00:00, 31.99it/s]


In [22]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,공장_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",0
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",1
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",1
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",1
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",1
...,...,...,...,...,...,...,...,...,...
15545,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",161
15546,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",206
15547,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",190
15548,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",187


In [24]:
sum_df['공장_개수'].value_counts()

0      2567
1      1550
2       759
3       582
4       498
       ... 
129       2
155       2
114       1
206       1
190       1
Name: 공장_개수, Length: 194, dtype: int64

In [25]:
sum_df.to_csv('공장개수_weight.csv', index=False)